# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
#print(classes)
for i in range(len(classes)):
    datatarget = load_pose_data(i)
    data = datatarget[0]
    target = datatarget[1]
    all_data = all_data + data
    all_target = all_target + target

#print ('total number of data', len(all_data))

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']
total number of data 222


In [60]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [61]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [62]:
# YOUR CODE HERE
xdata = []
ydata = []
for i in training_data:
    xdata.append(all_data[i])
    ydata.append(all_target[i])
clf.fit(xdata, ydata)

SVC(C=100.0, gamma=0.001)

### predicting

In [63]:
clf.predict(all_data[-1:]), all_target[-1]

(array([10]), 10)

In [64]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [65]:
expected = []
predicted = []
# YOUR CODE HERE
#over half of the test data
otherhalftest = permutation[n_training_data:]

for i in otherhalftest:
    expected.append([all_target[i]])
    predicted.append(clf.predict([all_data[i]]))

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         5
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         8
           7       1.00      0.80      0.89         5
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00        14

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.99        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 9  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [66]:
expected = []
predicted = []
# YOUR CODE HERE
#over half of the test data
otherhalftest = permutation[n_training_data:]

for i in otherhalftest:
    expected.append([all_target[i]])
    predicted.append(clf.predict([all_data[i]]))

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00         5
           2       0.91      1.00      0.95        10
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         8
           7       1.00      0.80      0.89         5
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00        14

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.99        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 9  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [67]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [68]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1]

(array([10]), 10)